In [205]:
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import random

path = "AmongUs_Dataset/"
files = [f for f in listdir(path) if isfile(join(path, f))]

In [240]:
Dataset = []
users_i = [0 for i in range(len(files))]

for i in range(len(files)):
#     print(files[i])
    file_path = "{}{}".format(path, files[i])
    d = pd.read_csv(file_path, delimiter=",")
    
    #add column for user id
    _id = int(files[i].split(".")[0][4:])-1
    users_i[_id] = int(len(d))
    user_id = [_id for n in range(len(d))]
    user_id_col = pd.Series(user_id, name="User_Id")
    user_data = pd.concat([d, user_id_col], axis=1)
    
    #add to compiled Dataset
    if len(Dataset) == 0:
        Dataset = user_data
    else:
        Dataset = Dataset.append(user_data)
Dataset.reset_index(drop=True, inplace=True)


In [241]:
Dataset

,Game Completed Date,Team,Outcome,Task Completed,All Tasks Completed,Murdered,Imposter Kills,Game Length,Ejected,Sabotages Fixed,Time to complete all tasks,Rank Change,Region/Game Code,User_Id
0,12/13/2020 at 1:26:56 am EST,Crewmate,Win,3,No,Yes,-,07m 04s,No,2.0,-,++,NA / WYMSBF,0
1,12/13/2020 at 1:17:42 am EST,Crewmate,Loss,7,Yes,No,-,16m 21s,No,1.0,09m 48s,--,NA / WYMSBF,0
2,12/13/2020 at 12:57:47 am EST,Crewmate,Win,3,No,No,-,11m 33s,No,0.0,-,++,NA / WYMSBF,0
3,12/13/2020 at 12:41:55 am EST,Imposter,Win,-,-,-,2,08m 05s,No,NaN,-,+++,Europe / QIRTNF,0
4,12/13/2020 at 12:30:37 am EST,Crewmate,Loss,4,No,No,-,05m 10s,No,0.0,-,---,Europe / QIRTNF,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,11/28/2020 at 12:57:06 pm EST,Imposter,Loss,-,-,-,2,11m 02s,No,NaN,-,-,Europe / DJCWJF,8
2223,11/28/2020 at 12:43:36 pm EST,Crewmate,Loss,7,Yes,Yes,-,06m 56s,No,0.0,04m 09s,--,Europe / DJCWJF,8
2224,11/28/2020 at 12:32:44 pm EST,Crewmate,Win,7,Yes,Yes,-,22m 06s,No,0.0,06m 31s,++,Europe / DJCWJF,8
2225,11/28/2020 at 12:09:02 pm EST,Crewmate,Loss,7,Yes,No,-,13m 41s,No,0.0,06m 06s,--,Europe / DJCWJF,8


In [242]:
def user_train_valid_split(users_i,valid_ratio,possible=False):
    n = sum(users_i)
    target = int(np.floor(valid_ratio*n))
    while not possible:
        possible, validation_indices = user_split(users_i,target)
        target -= 1
    training_indices = [i for i in range(len(users_i)) if i not in validation_indices]
    return training_indices, validation_indices

def user_split(users_i,target):  
    # Reference: https://levelup.gitconnected.com/dynamic-programming-subset-sum-c386126621cd
    n = len(users_i)
    solution = [[False for j in range(int(target+1))] for i in range(n+1)]
    # base cases
    for i in range(n):
        solution[i][0] = True
    # other cases
    for i in range(1,n+1):
        for j in range(1,target+1):
            solution[i][j] = solution[i-1][j]
            if(solution[i][j] == False and j >= users_i[i-1]):
                solution[i][j] = solution[i][j] or solution[i-1][j-users_i[i-1]]
    # check if the subset sum is possible
    possible = solution[len(users_i)][target]
    subset = []
    if not possible: return possible, subset
    # return the subset solution if one exists
    y = len(users_i)
    x = target
    while x != 0:
        if solution[y-1][x] == False:
            subset.append(y-1)
            x -= users_i[y-1]
        else: 
            y -= 1
    return possible, subset

In [243]:
valid_ratio = .25
training_indices, validation_indices = user_train_valid_split(users_i,valid_ratio)

print(users_i)
print(training_indices, validation_indices)
train_sum = sum([users_i[i] for i in training_indices])
valid_sum = sum([users_i[i] for i in validation_indices])
print(valid_sum/(train_sum+valid_sum))

[100, 100, 91, 100, 35, 100, 100, 100, 100, 100, 31, 100, 100, 100, 39, 70, 72, 100, 38, 64, 79, 100, 37, 38, 100, 33, 100, 54, 46]
[4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28] [19, 15, 10, 3, 2, 1, 0]
0.24966322406825325


In [247]:
train_data = Dataset.loc[Dataset['User_Id'].isin(training_indices)]
valid_data = Dataset.loc[Dataset['User_Id'].isin(validation_indices)]
train_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)

In [248]:
print(len(train_data), train_sum)

1671 1671
